# Tensorflow
* 1.图(Graph) : 用来表示计算任务,也就我们要做的一些操作.
* 2.会话(Session) : 建立会话,此时会生成一张空图;在会话中添加节点和边,形成一张图,一个会话可以有多个图,通过执行这些图得到结果.如果把每个图看做一个车床,那会话就是一个车间,里面有若干个车床,用来把数据生产成结果.
* 3.张量(Tensor) : 用来表示数据,是我们的原料.
* 4.变量(Variable) : 用来记录一些数据和状态,是我们的容器.
* 5.feed和fetch : 可以为任意的操作(arbitrary operation) 赋值或者从其中获取数据.相当于一些铲子,可以操作数据.

### 2.1 创建图和运行图

In [5]:
# 导入Tensorflow包
import tensorflow as tf

# 创建一个常量v1,它是一个1行2列的矩阵
v1 = tf.constant([[2, 3]])

In [6]:
# 创建一个常量v2,它是一个2行1列的矩阵
v2 = tf.constant([[2], [3]])

In [7]:
# 创建一个矩阵乘法,这里要注意的是,创建了乘法后,是不会立即执行的,要在session中执行才行
product = tf.matmul(v1, v2)

In [8]:
# 这个时候打印得到的不是乘法之后的结果,而是得到乘法本身
print(product)

Tensor("MatMul:0", shape=(1, 1), dtype=int32)


In [10]:
# 定义一个session
sess = tf.Session()
# 运算乘法,得到结果
result = sess.run(product)
# 打印结果
print(result)
# 关闭session
sess.close()

[[13]]


### 2.2 创建一个变量,并用for循环对变量进行赋值操作

In [11]:
# 创建一个变量num
num = tf.Variable(0, name = "count")
# 创建一个加法操作,把当前数字+10
new_value = tf.add(num, 10)
# 创建一个赋值操作,把new_value赋值给num
op = tf.assign(num, new_value)

# 使用这种写法在运行完毕后session会自动关闭
with tf.Session() as sess:
    # 初始化变量
    sess.run(tf.global_variables_initializer())
    # 打印最初num的值
    print(sess.run(num))
    # 创建一个for循环,每次给num+10,并打印出来
    for i in range(5):
        sess.run(op)
        print(sess.run(num))

0
10
20
30
40
50


### 2.3 通过feed设置placeholder的值
有的时候，我们会在声明变量的时候不赋值，计算的时候才进行赋值，这个时候feed就派上用场了

In [17]:
# 创建一个变量占位符input1
input1 = tf.placeholder(tf.float32)
# 创建一个变量占位符input2
input2 = tf.placeholder(tf.float32)

# 创建一个乘法操作,把input1和input2相乘
new_value = tf.multiply(input1, input2)

# 使用这种写法在运行完毕后session会自动关闭
with tf.Session() as sess:
    # 打印new_value的值,在运算时,用feed设置两个输入的值
    print(sess.run(new_value, feed_dict={input1: 23.0, input2: 11.0}))

253.0


### 3.1 载入数据，并对数据进行处理
在写代码的过程中，数据的预处理是最大的一块工作，做一个项目，60%以上的代码在做数据预处理。

这个项目的预处理，分为5步：

1）把输入和结果分开

2）对输入进行处理：把一维的输入变成28*28的矩阵

3）对结果进行处理：把结果进行One-Hot编码

4）把训练数据划分训练集和验证集

5）对训练集进行分批

In [21]:
# 导入需要的库
import numpy as np
import pandas as pd
import tensorflow as tf

In [34]:
# 1 加载数据集,把对输入和结果进行分开
train = pd.read_csv("train.csv")
images = train.iloc[:, 1:].values
labels_flat = train[[0]].values.ravel()

# 2 对输入进行处理
images = images.astype(np.float)
images = np.multiply(images, 1.0 / 255.0)
print("输入数据的数量: (%g, %g)" % images.shape)

image_size = images.shape[1]
print("输入数据的维度: {%s}" % image_size)

image_width = image_height = np.ceil(np.sqrt(image_size)).astype(np.uint8)
print("图片的长: {0}\n图片的高: {1}".format(image_width, image_height))

x = tf.placeholder("float", shape=[None, image_size])

# 3 对结果进行处理
labels_count = np.unique(labels_flat).shape[0]
print("结果的种类: {0}".format(labels_count))
# 进行One-hot编码
def dense_to_one_hot(labels_dense, num_classes):
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    return labels_one_hot

labels = dense_to_one_hot(labels_flat, labels_count)
labels = labels.astype(np.uint8)
print("结果的数量: ({0[0]}, {0[1]})".format(labels.shape))

# 4 把输入数据划分训练集和验证集
# 把40000个数据作为训练集,2000个数据作为验证集
VALIDATION_SIZE = 2000

validation_images = images[:VALIDATION_SIZE]
validation_labels = labels[:VALIDATION_SIZE]

train_images = images[VALIDATION_SIZE:]
train_lables - labels[VALIDATION_SIZE:]

# 5 对训练集进行分批
batch_size = 100
n_batch = len(train_images) / batch_size

KeyError: '[0] not in index'